<a href="https://colab.research.google.com/github/chojinsoo/Data-Engineering-Study/blob/main/(10%EA%B8%B0)_Transaction_%EB%B0%B0%EC%9A%B0%EA%B8%B0_Jinsoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 트랜잭션이란? 동시에 Atomic하게 실행되어야 하는 SQL들을 묶어서 하나의 작업처럼 처리하는 방법

In [ ]:
import psycopg2

# Redshift connection 함수
# - user와 password를 본인 것으로 수정!
def get_Redshift_connection(autocommit):
    host = "learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com"
    user = "chojs1016"
    password = "Chojs1016!1"
    port = 5439
    dbname = "dev"
    conn = psycopg2.connect(f"dbname={dbname} user={user} host={host} password={password} port={port}")
    conn.set_session(autocommit=autocommit)
    return conn

# INSERT SQL을 autocommit=False로 실행

In [ ]:
conn = get_Redshift_connection(False)
cur = conn.cursor()

In [ ]:
cur.execute("SELECT * FROM chojs1016.name_gender LIMIT 10;")
res = cur.fetchall()
for r in res:
  print(r)

('Adaleigh', 'F')
('Amryn', 'Unisex')
('Apurva', 'Unisex')
('Aryion', 'M')
('Alixia', 'F')
('Alyssarose', 'F')
('Arvell', 'M')
('Aibel', 'M')
('Atiyyah', 'F')
('Adlie', 'F')


In [ ]:
cur.execute("DELETE FROM chojs1016.name_gender;")

In [ ]:
cur.execute("INSERT INTO chojs1016.name_gender VALUES ('Jinsoo', 'Female');")

In [ ]:
cur.execute("SELECT * FROM chojs1016.name_gender LIMIT 10;")
res = cur.fetchall()
for r in res:
  print(r)

('Jinsoo', 'Female')


In [ ]:
cur.execute("COMMIT;")   # conn.commit()와 동일. cur.execute("ROLLBACK;")와 conn.rollback()도 동일

In [ ]:
conn.close()

# INSERT SQL을 autocommit=False로 실행하고 try/except로 컨트롤하기

In [ ]:
conn = get_Redshift_connection(False)
cur = conn.cursor()

In [ ]:
try:
  cur.execute("DELETE FROM chojs1016.name_gender;") 
  cur.execute("INSERT INTO chojs1016.name_gender VALUES ('Claire', 'Female');")
  conn.commit()  # cur.execute("COMMIT;")와 동일
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
  conn.rollback()  # cur.execute("ROLLBACK;")와 동일
finally :
  conn.close()

# INSERT SQL을 autocommit=True로 실행하고 try/except로 컨트롤하기

In [ ]:
conn = get_Redshift_connection(True)
cur = conn.cursor()

In [ ]:
cur.execute("DELETE FROM chojs1016.name_gender;") 

In [ ]:
try:
  cur.execute("BEGIN;")
  cur.execute("DELETE FROM chojs1016.name_gender;") 
  cur.execute("INSERT INTO chojs1016.name_gender VALUES ('Claire', 'Female');")
  cur.execute("END;")
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
  cur.execute("ROLLBACK;")
finally :
  conn.close()

# INSERT SQL을 autocommit=True로 실행하고 SQL로 컨트롤하기

In [ ]:
conn = get_Redshift_connection(True)
cur = conn.cursor()

In [ ]:
cur.execute("SELECT * FROM chojs1016.name_gender;")
res = cur.fetchall()
for r in res:
  print(r)

('Claire', 'Female')


In [ ]:
cur.execute("BEGIN;")
cur.execute("DELETE FROM chojs1016.name_gender;")
cur.execute("INSERT INTO chojs1016.name_gender VALUES ('Benjamin', 'Male');")

In [ ]:
cur.execute("END;")

# 잘못된 SQL을 중간에 실행해보기

In [ ]:
cur.execute("BEGIN;")
cur.execute("DELETE FROM chojs1016.name_gender;")
cur.execute("INSERT INTO chojs1016.name_gender3 VALUES ('Andrew', 'Male');")
cur.execute("END;")

UndefinedTable: ignored